In [3]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import os
import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text
import os
import uuid
import pandas as pd
from sqlalchemy import create_engine, text
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import os
import pandas as pd

load_dotenv()
engine = create_engine(os.getenv("DATABASE_URL"))

df = pd.read_sql("SELECT * FROM hpce.churn_model_dataset_multi", engine)

features = [
    "recency_days", "f_30", "m_30", "views_30", "events_30",
    "f_90", "m_90", "views_90", "events_90"
]

results = []

for label in ["churn_30d", "churn_60d", "churn_90d"]:
    X = df[features].fillna(0)
    y = df[label]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    model = XGBClassifier(
        n_estimators=100,
        max_depth=4,
        learning_rate=0.1,
        eval_metric="logloss",
        random_state=42
    )

    model.fit(X_train, y_train)
    preds = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, preds)

    results.append((label, auc))

results

[('churn_30d', 1.0),
 ('churn_60d', 0.9994584148990007),
 ('churn_90d', 0.9994322905050588)]